In [6]:

from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from typing import Optional
import logging
from backend.config import load_config

# Configuration du logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# %% [markdown]
# ## 2. Définition du modèle de données

# %%
class HeadData(BaseModel):
    nom: str = Field(description="Nom de famille du candidat")
    prenom: str = Field(description="Prénom du candidat") 
    email: Optional[str] = Field(description="Adresse email du candidat")
    telephone: Optional[str] = Field(description="Numéro de téléphone du candidat")
    linkedin: Optional[str] = Field(description="Lien LinkedIn du candidat")
    presentation: str = Field(description="Courte présentation/résumé du candidat")
    hobbies: Optional[list] = Field(description="Liste des hobbies/centres d'intérêt du candidat")

# %% [markdown]
# ## 3. Lecture du fichier source

# %%
with open('source_brut.txt', 'r', encoding='utf-8') as file:
    source_text = file.read()

print("Contenu du fichier source :")
print(source_text[:500] + "...") # Affiche les 500 premiers caractères




Contenu du fichier source :
Alexis de Monts
Data Engineer / Data Scientist | Expert Cloud et Data
Coordonnées : alexis.demonts.s@gmail.com 07 81 37 86 80 | LinkedIn
Résumé général
Fort d’une formation scientifique généraliste et d’une spécialisation en Data Science, je mets mes compétences au service de la transformation numérique des entreprises. Mon expertise couvre les domaines du Cloud (GCP, AWS), DevOps (CI/CD, Docker), MLOps, et l’intégration d’algorithmes de Machine Learning dans des solutions IT complexes.
Mes réal...


In [7]:
# %% [markdown]
# ## 4. Configuration du LLM et du parser

# Initialisation du modèle LLM
llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0
)

# Parser JSON
parser = JsonOutputParser(pydantic_object=HeadData)

# Template du prompt
prompt = PromptTemplate(
    template="""
    Analyse le texte suivant et extrait uniquement les informations personnelles du candidat.
    Ne prends pas en compte les expériences professionnelles ni la formation.
    
    {format_instructions}
    
    Texte source:
    {source}
    """,
    input_variables=["source"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

# %% [markdown]
# ## 5. Exécution de la chaîne et extraction des données

# %%
# Création de la chaîne
chain = prompt | llm | parser



In [9]:
# Exécution
head_data = chain.invoke({"source": source_text})

# %% [markdown]
# ## 6. Affichage des résultats

# %%
print("Données extraites du CV :")
for key, value in head_data.items():
    print(f"{key}: {value}")


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Données extraites du CV :
nom: de Monts
prenom: Alexis
email: alexis.demonts.s@gmail.com
telephone: 07 81 37 86 80
linkedin: www.linkedin.com/in/alexis-demonts-61328a175
presentation: Ingénieur des Ponts et Chaussées | Expert Cloud et Data. Fort d’une formation scientifique généraliste et d’une spécialisation en Data Science au Politecnico di Milano, j’aide les entreprises à transformer leurs données en valeur.
hobbies: None
